In [176]:
from bs4 import BeautifulSoup
import requests
from HTMLParser import HTMLParser
import json
import collections
import os
import itertools

In [220]:
#html = 'beer_styles/stout.html'
#html = 'beer_styles/wheatale.html'
html = 'beer_styles/belgianale.html'
with open(html) as file:
    soup = BeautifulSoup(file, ["lxml", "xml"])

<?xml version="1.0" encoding="utf-8"?>
<!-- saved from url=(0041)http://www.ratebeer.com/advbeersearch.php --><html><head><meta content="text/html; charset=windows-1252" http-equiv="Content-Type"/><body><div class="line-gutter-backdrop"/><table><tbody><tr><td class="line-number" value="1"/><td class="line-content"><span class="html-doctype">!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"</span>
</td></tr><tr><td class="line-number" value="2"/><td class="line-content">
</td></tr><tr><td class="line-number" value="3"/><td class="line-content"><span class="html-tag">html</span>
</td></tr><tr><td class="line-number" value="4"/><td class="line-content"><span class="html-tag">head</span>
</td></tr><tr><td class="line-number" value="5"/><td class="line-content"><span class="html-tag">title</span>RateBeer.com -- Advanced Beer Search Results<span class="html-tag">/title</span>
</td></tr><tr><td class="line-number" value="6"

In [251]:
def gen_beerlist_noA(html):
    with open(html) as file:
        soup = BeautifulSoup(file, ["lxml", "xml"])
    second_links = []
    for link in soup.find_all('a', class_='html-attribute-value html-external-link'):
        link_val = link.get('href')
        if link_val.find('/beer/') != -1 and link_val.find('distribution/') == -1:
            second_links.append(link.get('href'))
    return second_links[:-1]

testlinks = gen_beerlist_noA('beer_styles/belgianale.html')

In [178]:
result = soup.findAll('A')
result = result[1:] #first result isn't a beer url

link= result[0].get('HREF')
print link.split('/')

def extract_url(list_el):
    link = list_el.get('HREF')
    name = link.split('/')[2]
    return link, name


[u'', u'beer', u'samuel-adams-summer-ale', u'171', u'']


In [186]:
def gen_beerlist(html):
    with open(html) as file:
        soup = BeautifulSoup(file, ["lxml", "xml"])
    
    result = soup.findAll('A')
    result = result[1:] #first result isn't a beer url
    links = []
    for r in result:
        rlink, rname = extract_url(r)
        links.append(rlink)
    return links

In [190]:
links_beers = gen_beerlist(html)
print links_beers[1:5]
html_base = 'http://www.ratebeer.com'
print html_base + links_beers[1]
#r = requests.get(html_base+links[1]) --> this works

[u'/beer/bells-oberon-ale/3211/', u'/beer/three-floyds-gumballhead/22764/', u'/beer/goose-island-312-urban-wheat-ale/30913/', u'/beer/widmer-brothers-hefeweizen/606/']
http://www.ratebeer.com/beer/bells-oberon-ale/3211/


In [199]:
def extract_style(html):
    locate_style = html.find('Style:')
    subhtml = html[locate_style:]
    start = subhtml.find('">')
    end = subhtml.find('</a>')
    style = subhtml[start+len('">'):end]
    style = style.replace('<br>','')
    style = style.encode('ascii', 'ignore')
    return style

def extract_name(html):
    locate_name = html.find('TITLE>')
    subhtml = html[locate_name:]
    start = subhtml.find('<TITLE>')
    end = subhtml.find('</TITLE>')
    name = subhtml[start+len('<TITLE>'):end]
    name = name.replace('<br>','')
    name = name.encode('ascii', 'ignore')
    return name

def extract_brewery(html):
    locate_brewery = html.find('Brewed by')
    subhtml = html[locate_brewery:]
    start = subhtml.find('">')
    end = subhtml.find('</A>')
    brewery = subhtml[start+len('">'):end]
    brewery = brewery.replace('<br>','')
    brewery = brewery.encode('ascii', 'ignore')
    return brewery

def extract_descript(html):
    locate_descript = html.find('COMMERCIAL DESCRIPTION')  
    subhtml = html[locate_descript:]
    start = subhtml.find('</small>')
    end = subhtml.find('</div>')
    descript = subhtml[start + len('</small>'): end]
    descript = descript.replace('<br>','')
    descript = descript.replace('<br>','')
    descript = descript.encode('ascii', 'ignore')
    return descript



In [214]:
def extract_ratebeer(html):
    #return dictionary for that beer entry
    html_base = 'http://www.ratebeer.com'
    link_beer = html_base + html
    r = requests.get(link_beer)
    htmlbeer = r.text
    name = extract_name(htmlbeer)
    descr = extract_descript(htmlbeer)
    brewery = extract_brewery(htmlbeer)
    style = extract_style(htmlbeer)
    ratebeer_dict = {name: (descr, brewery, style, link_beer)}
    return ratebeer_dict



In [253]:
tmpdes = extract_ratebeer(links_beers[1])
print extract_ratebeer(testlinks[1])
print tmpdes

{'': ('', '', '', u'http://www.ratebeer.comhttp://www.ratebeer.com/beer/ommegang-rare-vos/8838/')}
{'Bells Oberon Ale': ('Bells Oberon is a wheat ale fermented with Bells signature house ale yeast, mixing a spicy hop character with mildly fruity aromas. The addition of wheat malt lends a smooth mouthfeel, making it a classic summer beer.', 'Bells Brewery', 'Wheat Ale', u'http://www.ratebeer.com/beer/bells-oberon-ale/3211/')}


In [216]:
def addtodict(dic, beerdic):
    beerdic.update(dic)

In [217]:
#stout = {}
wheatale = {}
for li in links_beers[:5]:
    print li
    tmp = extract_ratebeer(li)
    addtodict(tmp, wheatale)
    #addtodict(tmp[0], tmp[1], stout)

/beer/samuel-adams-summer-ale/171/
/beer/bells-oberon-ale/3211/
/beer/three-floyds-gumballhead/22764/
/beer/goose-island-312-urban-wheat-ale/30913/
/beer/widmer-brothers-hefeweizen/606/


In [218]:
print wheatale

{'Widmer Brothers Hefeweizen': ('A golden unfiltered wheat beer that is truly cloudy and clearly superb. ', 'Widmer Brothers Brewing Company', 'Wheat Ale', u'http://www.ratebeer.com/beer/widmer-brothers-hefeweizen/606/'), 'Three Floyds Gumballhead': ('An American Wheat Beer brewed with red wheat and boat loads of amarillo hops gives this summer brew a lemony finish.  The slight haze in the bottle is from the bottle-conditioning, where yeast is added to the bottle for a secondary fermentation.  Gumballhead is a Skingraft Comics hero.  Check out Skingraft Comics and Records at www.skingraftrecords.com.  Cheers&#033;', 'Three Floyds Brewing Company', 'Wheat Ale', u'http://www.ratebeer.com/beer/three-floyds-gumballhead/22764/'), 'Bells Oberon Ale': ('Bells Oberon is a wheat ale fermented with Bells signature house ale yeast, mixing a spicy hop character with mildly fruity aromas. The addition of wheat malt lends a smooth mouthfeel, making it a classic summer beer.', 'Bells Brewery', 'Wheat

In [194]:
#with open('datastout.txt','w') as outfile:
#    json.dump(stout, outfile)
    
    
with open('datawheatale.txt','w') as outfile:
    json.dump(wheatale, outfile)
    
#this saves as a dictionary, want to save each line different or 
#save in a different format